In [1]:
import tensorflow as tf
from tensorflow import keras

In [6]:
x = tf.random.normal((100, 5))
y = tf.random.normal((100, 1))

In [9]:
class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = keras.layers.Dense(64, activation='relu')
        self.d2 = keras.layers.Dense(1)

    def call(self, inputs):
        x = self.d1(inputs)
        return self.d2(x)

model = MyModel()
model.compile(optimizer='adam', loss='mse')
model.fit(x, y, epochs=10)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.2156 
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1798 
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1435 
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.1162 
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0921 
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0700 
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.0494 
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.0304
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.0107 
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9980 


In [11]:
class MyCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch} completed, loss = {logs['loss']}")


In [12]:
model.fit(x, y, epochs=5, callbacks=[MyCallback()])


Epoch 1/5
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 1.0542Epoch 0 completed, loss = 0.9808872938156128
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9809 
Epoch 2/5
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.7426Epoch 1 completed, loss = 0.9693384766578674
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.9693
Epoch 3/5
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 1.1600Epoch 2 completed, loss = 0.95806884765625
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9581 
Epoch 4/5
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.8626Epoch 3 completed, loss = 0.9488640427589417
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9489 
Epoch 5/5
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.9560Epoch 4 completed, loss = 0.9393197894096375
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.9393


## FULL WORKING EXAMPLE

In [13]:
import tensorflow as tf

class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.kernel = self.add_weight(
            name="kernel",
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform",
            trainable=True
        )

        self.bias = self.add_weight(
            name="bias",
            shape=(self.units,),
            initializer="zeros",
            trainable=True
        )

    def call(self, inputs):
        return tf.nn.relu(inputs @ self.kernel + self.bias)


In [14]:
class MyCustomModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.layer1 = MyDenseLayer(16)
        self.layer2 = MyDenseLayer(8)
        self.out = tf.keras.layers.Dense(1)   # regression output

    def call(self, inputs):
        x = self.layer1(inputs)
        x = self.layer2(x)
        return self.out(x)


In [15]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 2 == 0:
            print(f"Epoch {epoch} - Loss: {logs['loss']:.4f}")

        # early stopping condition
        if logs["loss"] < 0.01:
            print("Stopping early! Loss too small.")
            self.model.stop_training = True


In [16]:
x = tf.random.normal((500, 5))     # 500 samples, 5 features
y = tf.reduce_sum(x, axis=1, keepdims=True) + tf.random.normal((500, 1))

In [17]:
model = MyCustomModel()
model.compile(optimizer="adam", loss="mse")

history = model.fit(
    x, y,
    epochs=20,
    batch_size=32,
    callbacks=[MyCallback()]
)

Epoch 1/20
 1/16 ━━━━━━━━━━━━━━━━━━━━ 12s 812ms/step - loss: 9.7522Epoch 0 - Loss: 6.4561
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 6.4561   
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.1582 
Epoch 3/20
 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 6.5819Epoch 2 - Loss: 5.8394
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.8394 
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.5028 
Epoch 5/20
 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 4.2988Epoch 4 - Loss: 5.1173
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.1173 
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.6811 
Epoch 7/20
 1/16 ━━━━━━━━━━━━━━━━━━━━ 2s 145ms/step - loss: 4.3624Epoch 6 - Loss: 4.2056
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.2056  
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6555 
Epoch 9/20
 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 2.6528Epoch 8 - Loss: 3.0934
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.0934 
Epoch 10/20
16/